# Instruction Finetuning

## [Tópicos em Ciência de Dados](https://denmartins.github.io/teaching/2025-topicos-cd)

### [Prof. Dr. Denis Mayr Lima Martins](https://denmartins.github.io/)

### [Pontifícia Universidade Católica de Campinas](https://www.puc-campinas.edu.br/)

<img src="https://www.puc-campinas.edu.br/wp-content/uploads/2022/06/logo-puc.png" width="100px"/>


## Objetivos de Aprendizagem
---

*   Explicar a necessidade e função do Ajuste Fino de Instruções (AFI)
*   Descrever a metodologia de treinamento e perda: detalhar a estrutura dos dados de AFI (instrução, contexto, resposta alvo) e explicar a função da perda
*   Avaliar o desempenho do modelo: utilizar métricas de avaliação e entender o papel de modelos externos (*LLM-as-a-Judge*) na avaliação da qualidade do alinhamento.

<center>
<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
Baseado no Livro <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> de <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="200px"></a>
</td>
</tr>
</table>
</center>

## Relembrando: O Conceito de Fine-Tuning
---

*   **Definição:** É o processo de utilizar um modelo pré-treinado como base e treiná-lo adicionalmente em um *dataset* menor e específico de um domínio ou tarefa.
*   **Objetivo:** Adaptar o modelo ao novo contexto, aprimorando o desempenho em aplicações especializadas, como tradução de linguagem, análise de sentimento ou sumarização.
*   **Vantagem:** O *fine-tuning* se baseia no conhecimento pré-existente do modelo, o que reduz substancialmente os requisitos computacionais e de dados em comparação com o treinamento do modelo do zero (*pre-training*).


## Por Que os LLMs Tradicionais Falham com Diretivas
---

*   **Objetivo do Pré-treinamento:** Os LLMs são otimizados para o reconhecimento de padrões linguísticos, minimizando o erro de previsão contextual da próxima palavra em vastos *corpora*.
    *   O modelo prevê o próximo *token* em uma sequência com base em padrões estatísticos.
*   **A Limitação:** Este objetivo de previsão do próximo *token* não otimiza inerentemente o modelo para seguir instruções explícitas do usuário.
    *   Sem treinamento adicional, um LLM de base simplesmente *completa* um *prompt*, em vez de fornecer uma *resposta* útil.
    *   Exemplo: Solicitar "*me ensine a fazer pão*" pode resultar em "*em um forno de casa*" (uma conclusão gramaticalmente correta, mas inútil).
*   **A Solução:** O AFI refina o modelo pré-treinado para interpretar as consultas do usuário como instruções formais que exigem ações específicas.

## O Que é Ajuste Fino de Instruções (AFI)?
---

*   **Definição:** AFI é uma técnica de ajuste fino que refina LLMs pré-treinados para aderir a instruções de tarefas específicas.
*   **Metodologia:** Envolve treinamento supervisionado em conjuntos de dados que consistem em pares explícitos de *prompt*-resposta.
*   **Função Chave:** O AFI preenche a lacuna entre a capacidade inerente de previsão da próxima palavra do LLM e o objetivo definido pelo ser humano de aderir a diretivas.
*   **Benefícios:**
    1.  **Alinhamento:** Conecta o objetivo de pré-treinamento com o objetivo de seguir instruções.
    2.  **Controlabilidade:** Restringe os *outputs* do modelo para alinhá-los com as características desejadas (e.g., formato ou conhecimento de domínio).
    3.  **Generalização:** Modelos ajustados por instruções demonstram forte desempenho *zero-shot* e *few-shot* em tarefas não vistas.


## Anatomia de uma Amostra de Dados AFI
---

O AFI requer pares de instruções e seus *outputs* de alta qualidade correspondentes.

*   **1. Instrução (A Diretiva):** Define claramente a tarefa necessária.
    *   *Exemplo:* "Traduza a seguinte frase para o Francês".
*   **2. Input/Contexto (O Conteúdo):** Informações suplementares opcionais relevantes para a tarefa.
    *   *Exemplo:* "A frase a traduzir: 'O processo de ajuste fino é complexo.'".
*   **3. Resposta Alvo (A Resposta Ouro):** O *output* de referência de alta qualidade que demonstra a conclusão correta da tarefa.

## Prompt Style Template
---

<center>
<img src="https://camo.githubusercontent.com/a0d3a2f932be145f5afd61d2abf54db358353ed2d30b4f5a54c508ac01a65b74/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f30342e776562703f32" width="700px">
</center>

## Estratégias de Coleta de Dados I
---

A curadoria e o dimensionamento de pares de instrução-*output* de alta qualidade são desafios centrais do AFI.

*   **1. Dados Criados por Humanos:** Dados anotados manualmente ou obtidos diretamente, confiando apenas na coleta e verificação humana.
    *   **Prós:** Geralmente a mais alta qualidade e consistência.
    *   **Contras:** Demorado e custoso para grandes escalas.
    *   Exemplos: [Databricks Dolly](https://github.com/databrickslabs/dolly/tree/master/data) (15K instâncias, abrangendo 7 tipos como Q&A, escrita criativa), [Meta LIMA](https://arxiv.org/pdf/2305.11206) (1K [exemplos](https://huggingface.co/datasets/GAIR/lima) cuidadosamente selecionados).
*   **2. Integração de Dados de Conjuntos de Dados Anotados:**
    *   Envolve a conversão de conjuntos de dados de PLN existentes (e.g., NLI, análise de sentimentos) em pares de instrução-*output* usando *templates*.
    *   Isto formaliza diversas tarefas de PLN em um formato unificado *sequence-to-sequence*.
    *   Exemplos: [FLAN](https://github.com/google-research/FLAN/tree/main/flan/v2) (transforma 62 *benchmarks* de PLN), [P3](https://huggingface.co/datasets/bigscience/P3) (integra 170 conjuntos de dados de PLN).


## Estratégias de Coleta de Dados II
---

LLMs podem ser usados para aumentar conjuntos de dados de AFI quando a criação manual é inviável.
 
*   [**Self-Instruct**](https://arxiv.org/pdf/2212.10560): Começa com um pequeno conjunto de pares sementes. Um LLM gera novas instruções e outra instância gera *outputs* correspondentes.
    *   *Exemplo:* O modelo [Alpaca usou 52K pares sintéticos](crfm.stanford.edu/2023/03/13/alpaca.html) gerados desta forma.
*   [**Bonito**](https://arxiv.org/pdf/2402.18334): Converte texto não anotado em datasets de treino para AFI. Modelo base: Mistral-7B
*   [**Magpie**](https://arxiv.org/html/2406.08464v2): Gera dados de instrução solicitando a um LLM alinhado (e.g., Llama 3 8B Instruct) com um *template* de pré-consulta para sintetizar instruções e respostas de forma totalmente automática.
<!-- *   **Autoaperfeiçoamento (Evol-Instruct):** Aumenta a complexidade da instrução.
    *   **Evolução em Profundidade:** Solicita a um LLM injetar restrições (e.g., limites de palavras) ou aumentar os passos de raciocínio.
    *   **Evolução em Amplitude:** Solicita ao LLM gerar instruções totalmente novas em domínios sub-representados. -->

## Alpaca
---

<div style='align: left; text-align:center;'>
    <img src='https://api.wandb.ai/files/capecape/images/projects/38233410/f8eba1d5.png' alt='Visão Geral Alpaca' style="width:800px;"/>
    <span style='display:block;'>Workflow Alpaca. AFI sobre o modelo base Llama-7B. Qualidade similar ao modelo da OpenAI, mas muito menor e mais barato de reproduzir. Fonte: <a href="https://wandb.ai/capecape/alpaca_ft/reports/How-to-fine-tune-an-LLM-Part-1-Preparing-a-Dataset-for-Instruction-Tuning--Vmlldzo1NTcxNzE2" target="_blank">Weights and Biases</a>.</span>
    <br/>
</div>

## Bonito
---

<div style='align: left; text-align:center;'>
    <img src='https://towardsdatascience.com/wp-content/uploads/2024/03/06Y_mH9Oik8938xgu.png' alt='Visão Geral Bonito' style="width:700px;"/>
    <span style='display:block;'>Workflow do framework Bonito. Fonte: <a href="https://arxiv.org/pdf/2402.18334" target="_blank">Learning to Generate Instruction Tuning Datasets for Zero-Shot Task Adaptation</a>.</span>
    <br/>
</div>

## Magpie
---

<div style='align: left; text-align:center;'>
    <img src='https://raw.githubusercontent.com/magpie-align/magpie/main/figs/overview.png' alt='Visão Geral Magpie' style="width:800px;"/>
    <span style='display:block;'>Workflow do framework Magpie. Step 1: apenas pre-query template como entrada para LLM e geração autorregressiva de instrução. Step 2: Combinação de post-query template e outra pre-query template. Fonte: <a href="https://arxiv.org/html/2406.08464v2" target="_blank">Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing</a>.</span>
    <br/>
</div>

## Pré-processamento
---

1.  **Formatação do Prompt:** Adote um estilo de *prompt* consistente (e.g., Alpaca) para todas as amostras de treinamento.
2.  **Tokenização:** Converta o texto formatado de instrução-resposta em IDs de *token*.
3.  **Colagem/Preenchimento Customizado (packing):** Uma função de colagem customizada é usada para preencher sequências dentro de um lote (ou *batch*) até o comprimento da sequência mais longa nesse lote.
4.  **Mascaramento de Instrução (Opcional):** Mascarar IDs de *token* que correspondem à instrução impede que a função de perda seja calculada sobre o texto da instrução. Isto força o modelo a focar o treinamento na geração da *resposta*. Mas... (veja figura ao lado).

<center>

<img src='https://substackcdn.com/image/fetch/$s_!mC_S!,w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fa5293f2b-4861-42d3-a921-bbd4fbc2afc1_1600x684.png' alt='Instruction Masking x Instruction Modeling' style="width:500px;"/>
<span style='display:block;'>Instruction Masking x Instruction Modeling (não mascara instrução) em (<a href="https://arxiv.org/html/2405.14394v2" target="_blank">https://arxiv.org/html/2405.14394v2</a>). Fonte: <a href="https://magazine.sebastianraschka.com/p/llm-research-insights-instruction" target="_blank">Raschka</a>.</span>
<br/>

</center>

## Packing
---

<center>
<table style="width:100%;border:none;">
<tr>
<td style="vertical-align:middle;text-align:center;border:none;">
<img src='https://lweitkamp.github.io/posts/packing/packed_sequences.png' alt='Visão Geral Packing 1' style="width:800px;"/>
    <span style='display:block;'>Packing: Combinando múltiplas amostras em uma única sentença. Fonte: <a href="https://lweitkamp.github.io/posts/packing/index.html" target="_blank">Laurens Weitkamp</a>.</span>
    <br/>
</td>
<td style="vertical-align:middle;text-align:center;border:none;">
    <img src='https://api.wandb.ai/files/capecape/images/projects/38233410/d9f4c0c2.png' alt='Visão Geral Packing 2' style="width:800px;"/>
    <span style='display:block;'>Packing: Otimização do tamanho do contexto. Fonte: <a href="https://wandb.ai/capecape/alpaca_ft/reports/How-to-fine-tune-an-LLM-Part-1-Preparing-a-Dataset-for-Instruction-Tuning--Vmlldzo1NTcxNzE2" target="_blank">Weights and Biases</a>.</span>
    <br/>
</td>
</tr>
</table>
</center>


## Eficiência: Ajuste Fino com Eficiência de Parâmetros (PEFT)
---

O Ajuste Fino Completo (*Full Fine-Tuning*) é caro e corre o risco de *esquecimento catastrófico*. Os métodos PEFT reduzem drasticamente os custos.

*   **Low-Rank Adaptation (LoRA):** A técnica PEFT mais comum.
    *   Mantém a maioria dos parâmetros LLM pré-treinados congelados.
    *   Injeta pequenas matrizes de decomposição de baixa classificação ($A$ e $B$) nos parâmetros de atenção.
    *   Reduz drasticamente o número de parâmetros treináveis (e.g., 10.000x de redução para GPT-3) e o uso de memória.
*   **Quantized LoRA (QLoRA):** Uma extensão do LoRA otimizando ainda mais a memória.
    *   Quantiza os pesos base do LLM congelado para precisão ultrabaixa (e.g., 4-bit).
    *   Permite o ajuste fino de modelos de alta qualidade usando uma única GPU de consumo.
* Veja também o vídeo no Youtube: [LoRA explained (and a bit about precision and quantization)](https://www.youtube.com/watch?v=t509sv5MT0w).



## LoRA
---

<div style='align: left; text-align:center;'>
    <img src='https://substackcdn.com/image/fetch/$s_!Fk3V!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fee7f7d37-3c0a-4f4a-9244-f73287af6211_1456x612.jpeg' alt='Visão Geral LoRa' style="width:800px;"/>
    <span style='display:block;'>Finetuning convencional x finetuning LoRA. Fonte: <a href="https://arxiv.org/html/2406.08464v2" target="_blank">Raschka</a>.</span>
    <br/>
</div>

In [ ]:
import torch.nn as nn

class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

In [ ]:
class LinearWithLoRA(nn.Module):

    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)




## LoRA
---

<div style='align: left; text-align:center;'>
    <img src='https://lightningaidev.wpengine.com/wp-content/uploads/2023/04/lora-5-1536x766.png
' alt='Resultados LoRA' style="width:800px;"/>
    <span style='display:block;'>Resultados LoRA. Fonte: <a href="https://lightning.ai/pages/community/tutorial/lora-llm/" target="_blank">Lightning AI</a>.</span>
    <br/>
</div>

## Métricas de Avaliação I: Quantitativas e Técnicas

A avaliação mede o quão bem o modelo ajustado fino generaliza e adere aos objetivos. Leia mais em [Patterns for Building LLM-based Systems & Products](https://eugeneyan.com/writing/llm-patterns/).

*   **Cross-Entropy Loss (Perda de Entropia Cruzada):** A métrica fundamental monitorada durante o treinamento e a validação.
    *   Quantifica a diferença entre a distribuição de probabilidade prevista pelo modelo e a distribuição real de *tokens*.
*   **Métricas de PLN Tradicionais (para tarefas específicas):**
    *   **BLEU:** Mede a proximidade entre traduções geradas e de referência (Tradução Automática, Sumarização).
    *   **Acurácia/F1 Score:** Usado para tarefas de classificação e QA.
*   **Avaliação de Codificação:**
    *   [**HumanEval**](https://github.com/openai/human-eval): Consiste em 164 problemas de programação para avaliar a capacidade do modelo de gerar programas corretos a partir de *docstrings*.
*   **Aderência à Instrução:**
    *   [**IFEval (Instruction Following Evaluation)**](https://arxiv.org/pdf/2311.07911): Testa especificamente a capacidade de um modelo de seguir restrições explícitas, como contagem de palavras ou formatação de *output* necessária.

## Métricas de Avaliação II: Alinhamento e Pontuação
---

Para geração aberta, as métricas puramente automatizadas geralmente são insuficientes, exigindo avaliação centrada no ser humano.

*   **LLM-como-Juiz (*LLM-as-a-Judge*):** Utiliza um LLM altamente capaz (e.g., Llama 3 8B) para avaliar a qualidade dos *outputs*.
    *   O modelo Juiz recebe o *input*, o *output* correto e a resposta do modelo ajustado fino, fornecendo uma pontuação numérica (e.g., 0 a 100).
    *   Isto é eficiente para avaliação em grande escala.
*   **Benchmarking de Alinhamento:**
    *   **MT-Bench:** Usa 80 questões multi-turno de alta qualidade para avaliar o alinhamento com a preferência humana, cobrindo tarefas como escrita, codificação e raciocínio.
    *   **WildBench:** Curado a partir de interações reais do usuário, apresentando 1.024 instruções desafiadoras que exigem pensamento crítico.
*   **Métricas de Segurança:** Avaliam respostas do LLM quanto a toxicidade, viés e aderência a diretrizes de segurança. Exemplos incluem Llama Guard 2/3 e ShieldGemma.

## Armadilhas Comuns e Limitações em AFI
---

O AFI está sujeito a modos de falha específicos que podem minar a utilidade a longo prazo.

*   **Esquecimento Catastrófico (*Catastrophic Forgetting*):** O ajuste fino em novas tarefas pode fazer com que o modelo perca o conhecimento pré-treinado.
    *   *Mitigação:* Usar técnicas PEFT (LoRA/QLoRA) para congelar a maioria dos pesos base.
*   **Alinhamento Superficial (*Superficial Alignment*):** O modelo aprende apenas padrões de superfície e estilos (e.g., formato de *output* ou tom) em vez de melhorar o raciocínio subjacente.
    *   Isto levanta a preocupação de que os ganhos de desempenho dependam fortemente das tarefas representadas no dado de treinamento.
*   **Dependência da Qualidade dos Dados:** O desempenho depende criticamente da qualidade, diversidade e cobertura de tarefas do conjunto de dados de instrução.
    *   Dados mal selecionados (especialmente sintéticos) podem reforçar vieses ou deficiências.

## Resumo e Leitura Adicional
---

*   O AFI (SFT) é essencial para alinhar a previsão do próximo *token* dos LLMs com os objetivos do usuário.
*   O AFI depende de pares de instrução-resposta de alta qualidade e diversificados, gerados manualmente (Flan, Dolly) ou sinteticamente (Self-Instruct, Evol-Instruct).
*   O processo de treinamento usa uma perda de objetivo duplo e se beneficia de aprimoramentos arquiteturais (e.g., arquitetura de dois fluxos) e PEFT (LoRA/QLoRA).
*   A avaliação requer métricas quantitativas (Entropia Cruzada, HumanEval) e técnicas qualitativas (LLM-como-Juiz, checagens de segurança).
* **Leitura Adicional**:
    * [Instruction Pretraining LLMs](https://magazine.sebastianraschka.com/p/instruction-pretraining-llms?utm_source=publication-search)
    * [Instruction Tuning for Large Language Models: A Survey](https://arxiv.org/pdf/2308.10792)
    * [The Ultimate Guide to Fine-Tuning LLMs from Basics to Breakthroughs: An Exhaustive Review of Technologies, Research, Best Practices, Applied Research Challenges and Opportunities](https://arxiv.org/html/2408.13296v1)

## 1. Preparação de um Conjunto de Dados para Ajuste Fino Supervisionado por Instruções
---

Nesta seção, realizamos o download e a formatação do conjunto de dados de instrução destinado ao ajuste fino (finetuning) supervisionado de um modelo LLM pré‑treinado. O dataset contém 1 100 pares de *instruction–response* semelhantes aos apresentados anteriormente. Ele foi criado especificamente para este notebook; entretanto, existem outros conjuntos de dados de instruções disponíveis publicamente.

O código a seguir implementa e executa uma função que baixa esse conjunto de dados — um arquivo relativamente pequeno, com apenas 204 KB, em formato JSON. O JSON (JavaScript Object Notation) espelha a estrutura dos dicionários Python, oferecendo uma representação simples e legível por humanos e também adequada para intercâmbio de dados entre máquinas.


In [ ]:
import json
import os
import urllib

def download_and_load_file(file_path, url):

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
                file.write(text_data)
    else:                                     # Skip download if file was already downloaded
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data


file_path = "instruction-data.json"   # prepared by Sebastian Raschka (in Alpaca format)
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

In [ ]:
print("Example entry:\n", data[50])

In [ ]:
print("Another example entry:\n", data[999])

**Ajuste fino por instruções** costuma ser chamado de *“supervised instruction finetuning”* porque envolve treinar um modelo em um conjunto de dados no qual os pares entrada‑saída são explicitamente fornecidos. Existem diferentes formas de formatar as entradas como input para o LLM; a figura abaixo ilustra dois formatos utilizados, respectivamente, nos modelos Alpaca (https://crfm.stanford.edu/2023/03/13/alpaca.html) e Phi‑3 (https://arxiv.org/abs/2404.14219):


<center>
<img src="https://camo.githubusercontent.com/a0d3a2f932be145f5afd61d2abf54db358353ed2d30b4f5a54c508ac01a65b74/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f30342e776562703f32" width="700px">
</center>

> O estilo Alpaca (esquerda) emprega um formato estruturado com seções definidas para instrução, input e resposta;  
> O estilo Phi‑3 (direita) utiliza um formato mais simples com tokens designados <|user|> e <|assistant|>.


Utilizamos a formatação de prompt no estilo Alpaca, que foi o template original para ajuste fino por instruções. A seguir, formatamos a entrada que será enviada ao LLM.

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [ ]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"

print(model_input + desired_response)

In [ ]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[999]['output']}"

print(model_input + desired_response)

In [ ]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.1)    # 10% for testing
val_portion = len(data) - train_portion - test_portion  # Remaining 5% for validation

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [ ]:
print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

## 2. Organização dos Dados em Batches de Treino
---

Ao avançarmos na fase de implementação do processo de ajuste fino supervisionado por instruções, o próximo passo concentra-se na construção eficiente dos batches de treino. Isto requer definir um método que assegure que o modelo receba os dados formatados adequadamente durante o fine‑tuning.

Em ajuste fino para classificação, os batches eram gerados automaticamente pela classe `DataLoader` do PyTorch, a qual utiliza uma função *collate* padrão para combinar listas de amostras em batches. A função *collate* é responsável por receber uma lista de dados individuais e consolidá‑la num único batch que pode ser processado eficientemente pelo modelo.

Para ajuste fino supervisionado por instruções, o processo de batching envolve etapas adicionais: precisamos criar nossa própria função *collate* customizada, que será posteriormente inserida no `DataLoader`. Implementaremos essa função para atender às exigências específicas e à formatação do nosso conjunto de dados de instruções.


Dividimos a implementação do batching em cinco passos (conforme ilustrado na figura abaixo):

</br>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/305d903dda7544affa56b3f4e7c09adb2e2a1b65be56f40325422c1af0086d7e/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f30362e776562703f31' alt= '' style="height:550px;"/>
    <br/>
</div>

Primeiro, implementamos a classe `InstructionDataset`, que pré‑tokeniza todas as entradas do dataset — de forma análoga à `SpamDataset` usada em classificação — passando do formato JSON ao texto e, daí, aos IDs dos tokens. Este processo de dois passos ocorre no construtor `__init__`.

</br>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/ffb7d4ced87d4044dd261909e8acff9155182556cc1ecfea0babb225a5e6e3a2/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f30372e776562703f31' alt= '' style="height:550px;"/>
    <br/>
</div>


In [ ]:
import torch
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        # Pre-tokenize texts
        self.encoded_texts = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)

Aqui, nós coletamos múltiplos exemplos de treinamento em um só batch para acelerar o treinamento. Isso requer padding em todas as entradas para gerar tamanhos similares e adicionar `<|endoftext|>` como token de padding.

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))


Em ajuste fino por classificação, todos os exemplos eram padronizados para o mesmo comprimento globalmente. Aqui adotamos uma abordagem mais refinada: criamos uma função *collate* customizada que recebe cada batch individualmente e adiciona padding apenas até a maior sequência presente naquele batch (não em todo o dataset). Isto minimiza o padding desnecessário, pois batches diferentes podem ter comprimentos distintos.

</br>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/8ee26bf16d82ca4bcd948ba39de7cab6360980923bb66a2669c10d59af524fda/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f30382e776562703f31' alt= '' style="height:450px;"/>
    <br/>
</div>

A figura abaixo demonstra como os exemplos de treino são padronizados dentro de um batch usando o token ID 50256 para garantir comprimento uniforme. Cada batch pode ter tamanho diferente, como ilustrado nos dois primeiros batches da figura.

Podemos implementar esse processo de padding com a função *collate* customizada que operará sobre cada batch (a *collate* cuidará das etapas 2.3 a 2.5; a implementação será feita em vários rascunhos):

In [ ]:
def custom_collate_draft_1(           # just taking care of step 2.3
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # Find the longest sequence in the batch and increase the max length by +1, which will add one extra
    # padding token below
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs
    inputs_lst = []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to batch_max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        # Via padded[:-1], we remove the extra padded token
        # that has been added via the +1 setting in batch_max_length
        # (the extra padding token will be relevant in later codes)
        inputs = torch.tensor(padded[:-1])
        inputs_lst.append(inputs)

    # Convert list of inputs to tensor and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    return inputs_tensor


Esta função será posteriormente passada ao `DataLoader` para gerar batches adequados ao ajuste fino supervisionado por instruções.

In [ ]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_draft_1(batch))

Esta figura ilustra o alinhamento entre os tokens de entrada e os tokens alvo utilizados no processo de ajuste fino por instruções de um LLM.

</br>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/526e35a8193b42968ab498d636171ec8568524a0b6d5726e9a0a5d4bdf73c5ef/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f31302e776562703f31' alt= '' style="height:650px;"/>
    <br/>
</div>

In [ ]:
def custom_collate_draft_2(
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets
        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs to tensor and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)
    return inputs_tensor, targets_tensor

In [ ]:
inputs, targets = custom_collate_draft_2(batch)
print(inputs)
print(targets)

No passo seguinte, atribuímos ao valor placeholder –100 a todos os tokens de padding, conforme ilustrado abaixo. Esse valor especial permite que excluamos esses tokens de padding do cálculo da perda de treinamento, garantindo que apenas dados significativos influenciem o aprendizado do modelo.

</br>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/190ff066e25553da764adce88f5ab471309504bc23a3af43c0e303186b41c93a/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f31322e776562703f32' alt= '' style="height:350px;"/>
    <br/>
</div>

Depois de criar a sequência alvo deslocando os IDs dos tokens uma posição para a direita e acrescentando um token de fim‑de‑texto, o passo 2.5 concentra‑se em substituir os tokens de padding de fim‑de‑texto por um valor placeholder (–100). Observe que mantemos um token de fim‑de‑texto (ID 50256) na lista alvo. Isso permite que o LLM aprenda quando gerar um token de fim‑de‑texto em resposta às instruções, o qual usamos como indicativo de que a resposta gerada está completa. Concretamente, isso significa substituir os IDs dos tokens correspondentes a 50256 por –100, conforme ilustrado acima.

A figura mostra a substituição de todas as ocorrências do token de fim‑de‑texto, exceto a primeira (que usamos como padding), pelo valor placeholder –100, mantendo o token inicial de fim‑de‑texto em cada sequência alvo.

No trecho de código abaixo, modificamos nossa função de *collate* personalizada para substituir os tokens com ID 50256 por –100 nas listas de destino. Além disso, introduzimos o parâmetro `allowed_max_length` que permite limitar opcionalmente o tamanho das amostras. Essa alteração será útil caso você planeje trabalhar com seus próprios conjuntos de dados que excedam o limite de 1 024 tokens suportado pelo modelo GPT‑2. O código da função *collate* atualizada fica assim:

In [ ]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,  # a default value that cross-entropy loss will ignore
    allowed_max_length=None,  # truncate in case we have inputs exceeding the context length that the model supports
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index  # insert -100

        # New: Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor

In [ ]:
inputs, targets = custom_collate_fn(batch)
print(inputs)
print(targets)

O primeiro tensor representa as inputs e o segundo representa os targets.

Para testar, vamos ver como a troca para -100 ocorre. Vamos assumir uma pequena tarefa de classificação com 2 classes (0 e 1):

In [ ]:
logits_1 = torch.tensor(
    [[-1.0, 1.0],  # 1st training example
     [-0.5, 1.5]]  # 2nd training example
)
targets_1 = torch.tensor([0, 1])

z
loss_1 = torch.nn.functional.cross_entropy(logits_1, targets_1)
print(loss_1)

Agora, se adicionarmos mais um exemplo de treino, a loss será influenciada:

In [ ]:
logits_2 = torch.tensor(
    [[-1.0, 1.0],
     [-0.5, 1.5],
     [-0.5, 1.5]]  # New 3rd training example
)
targets_2 = torch.tensor([0, 1, 1])

loss_2 = torch.nn.functional.cross_entropy(logits_2, targets_2)
print(loss_2)


Vamos ver o que acontece se trocarmos o label da class pro um dos exemplos com -100:

In [ ]:
targets_3 = torch.tensor([0, 1, -100])

loss_3 = torch.nn.functional.cross_entropy(logits_2, targets_3)
print(loss_3)
print("loss_1 == loss_3:", loss_1 == loss_3)

O cálculo da loss ignorou o exemplo com label -100. Por padrão, o PyTorch possui a configuração cross_entropy(..., ignore_index=-100) que ignora exemplos correspondentes ao rótulo –100. Usando esse índice de ignorar –100, podemos descartar os tokens adicionais de fim‑de‑texto (padding) nos batches que usamos para padronizar os exemplos de treino a um mesmo comprimento. No entanto, desejamos manter um ID 50256 (fim‑de‑texto) nas metas porque isso ajuda o LLM a aprender a gerar tokens de fim‑de‑texto, os quais podemos usar como sinal de que uma resposta está completa.

## 3. Criando Data Loaders para um Dataset de Instruções

---

Até agora percorremos diversas etapas para implementar a classe `InstructionDataset` e a função `custom_collate_fn` para o dataset de instruções. Nesta seção, podemos colher os frutos do nosso trabalho simplesmente conectando ambos os objetos `InstructionDataset` e a função `custom_collate_fn` aos loaders de dados do PyTorch. Esses loaders irão embaralhar e organizar automaticamente os lotes (batches) para o processo de fine‑tuning de instruções no LLM.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")

print("Device:", device)

Um detalhe da função `custom_collate_fn` anterior é que agora movemos os dados diretamente para o dispositivo alvo (por exemplo, GPU), em vez de fazê‑lo dentro do loop principal de treinamento. Isso aumenta a eficiência porque pode ser executado como um processo em segundo plano quando usamos `custom_collate_fn` como parte do data loader.

Usando a função `partial` da biblioteca padrão `functools` do Python, criamos uma nova função com o argumento `device` pré‑preenchido. O código abaixo inicializa a variável de dispositivo:

In [ ]:
from functools import partial

customized_collate_fn = partial(
    custom_collate_fn,
    device=device,
    allowed_max_length=1024
)

In [ ]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_dataset = InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn, # here we plug in our customized version
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

In [ ]:
val_dataset = InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

Vamos verificar as dimensões de inputs e targets gerados pelo dataloader de treino:

In [ ]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Na saída, podemos observar que o primeiro lote de entrada e alvo possui dimensões de 8×61, onde 8 representa o tamanho do lote (batch size) e 61 é o número de tokens em cada exemplo de treinamento neste lote. O segundo lote de entrada e alvo possui um número diferente de tokens, por exemplo, 76. Todos os lotes possuem um tamanho de lote de 8, mas um comprimento diferente, como esperado.

Graças à nossa função `collate` personalizada, o data loader é capaz de criar lotes de comprimentos diferentes. Na próxima seção, carregaremos um LLM pré-treinado que poderemos então ajustar (finetune) com este data loader.


Vamos também verificar se as entradas contêm os tokens de preenchimento `<|endoftext|>` correspondentes ao ID do token 50256, imprimindo o conteúdo do primeiro exemplo de treinamento no último lote `inputs`:

In [ ]:
print(inputs[0])

Verificamos se os targets incluem -100:

In [ ]:
print(targets[0])

## 4. Carregando um LLM Pré-treinado
---

Acima, dedicamos muito tempo preparando o conjunto de dados para o ajuste fino (finetuning) por instrução, que é um aspecto fundamental do processo de ajuste supervisionado. Muitos outros aspectos são os mesmos que no pré-treinamento, permitindo-nos reutilizar grande parte do código de episódios anteriores. Especificamente, carregamos um modelo GPT pré-treinado usando o mesmo código de episódios anteriores, que serve como base para o treinamento subsequente. Este modelo pré-treinado, tendo já aprendido padrões e conhecimentos gerais da linguagem a partir de vastas quantidades de dados textuais, é então adaptado para seguir instruções através do processo de ajuste fino.

No entanto, em vez de carregar o menor modelo com 124 milhões de parâmetros, carregamos a versão média com 355 milhões de parâmetros, pois o modelo de 124 milhões é pequeno demais para alcançar resultados qualitativamente razoáveis através do ajuste fino por instrução. Isso é feito usando o mesmo código dos dois episódios anteriores, exceto que agora especificamos "gpt2-medium (355M)" em vez de "gpt2-small (124M)", ou seja, estamos usando um modelo maior para obter melhores resultados.

Observe que a execução do código fornecido abaixo iniciará o download da versão média do modelo GPT, que tem um requisito de armazenamento de aproximadamente 1,42 gigabytes. Este é mais ou menos três vezes maior do que o espaço de armazenamento necessário para o modelo pequeno:


In [ ]:
# These are the same definitions we have used before:

import os
import requests
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm


def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path)

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json")))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params


def download_file(url, destination):
    # Send a GET request to download the file in streaming mode
    response = requests.get(url, stream=True)

    # Get the total file size from headers, defaulting to 0 if not present
    file_size = int(response.headers.get("content-length", 0))

    # Check if file exists and has the same size
    if os.path.exists(destination):
        file_size_local = os.path.getsize(destination)
        if file_size == file_size_local:
            print(f"File already exists and is up-to-date: {destination}")
            return

    # Define the block size for reading the file
    block_size = 1024  # 1 Kilobyte

    # Initialize the progress bar with total file size
    progress_bar_description = url.split("/")[-1]  # Extract filename from URL
    with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
        # Open the destination file in binary write mode
        with open(destination, "wb") as file:
            # Iterate over the file data in chunks
            for chunk in response.iter_content(block_size):
                progress_bar.update(len(chunk))  # Update progress bar
                file.write(chunk)  # Write the chunk to the file


def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params


In [ ]:
from LLM_Definitions import GPTModel, load_weights_into_gpt

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(
    model_size=model_size,
    models_dir="gpt2"
)

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();


Antes de começarmos o ajuste fino do modelo, vamos ver como ele se comporta em uma das tarefas de validação comparando sua saída com a resposta esperada. Isso nos dará um entendimento inicial de quão bem o modelo se comporta em uma tarefa de seguimento de instruções logo de cara, antes do ajuste fino, e nos ajudará a apreciar o impacto do ajuste fino mais tarde.


In [ ]:
torch.manual_seed(123)

input_text = format_input(val_data[0])
print(input_text)


Em seguida, geramos a resposta do modelo usando a função `generate` que já implementamos:

In [ ]:
from LLM_Definitions import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

token_ids = generate(
    model=model,
    idx=text_to_token_ids(input_text, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256,
)
generated_text = token_ids_to_text(token_ids, tokenizer)

Note que a função `generate` retorna o texto de entrada e saída combinados. Esse comportamento foi conveniente em notebooks anteriores, já que os LLMs pré-treinados são projetados principalmente como modelos de completação de texto, onde a entrada e a saída são concatenadas para criar um texto coerente e legível. No entanto, ao avaliar o desempenho do modelo em uma tarefa específica, muitas vezes queremos nos concentrar apenas na resposta gerada pelo modelo.

Para isolar o texto da resposta do modelo, precisamos subtrair o comprimento da instrução de entrada do início do `generated_text`:

In [ ]:
response_text = (
    generated_text[len(input_text):]
    .replace("### Response:", "")
    .strip()
)
print(response_text)


Este trecho de código remove o texto da entrada do início do `generated_text`, deixando-nos apenas com a resposta gerada pelo modelo. A função `strip()` é então aplicada para remover quaisquer caracteres de espaço em branco à esquerda ou à direita.

Como podemos ver, o modelo ainda não é capaz de seguir as instruções; ele cria uma seção "Resposta", mas simplesmente repete a frase original, bem como a instrução.

## 5. Finetuning com Instruction Data
---


Agora, focamos no ajuste fino (finetuning) do modelo. Pegamos o modelo pré-treinado carregado na seção anterior e o treinamos ainda mais usando o conjunto de dados de instruções preparado anteriormente.

Já fizemos todo o trabalho difícil quando implementamos o processamento do conjunto de dados de instruções no início deste notebook. Para o processo de ajuste fino em si, podemos reutilizar a função de cálculo da perda e as funções de treinamento implementadas durante o pré-treino:

In [ ]:
from LLM_Definitions import (
    calc_loss_loader,
    train_model_simple
)


Vamos calcular a perda inicial do conjunto de treinamento e validação antes de começar o treinamento (como em episódios anteriores, o objetivo é minimizar a perda).

In [ ]:
model.to(device)

torch.manual_seed(123)

with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Note que o treinamento é um pouco mais caro do que em episódios anteriores, já que estamos usando um modelo maior (355 milhões de parâmetros em vez de 124 milhões). Os tempos de execução para vários dispositivos são mostrados abaixo para referência (executar este notebook em um dispositivo GPU compatível não requer alterações no código):

<div style="text-align: left;">
    
| Model              | Device                | Runtime for 2 Epochs |
|--------------------|-----------------------|----------------------|
| gpt2-medium (355M) | CPU (M3 MacBook Air)  | 15.78 minutes        |
| gpt2-medium (355M) | GPU (M3 MacBook Air)  | 10.77 minutes        |
| gpt2-medium (355M) | GPU (L4)              | 1.83 minutes         |
| gpt2-medium (355M) | GPU (A100)            | 0.86 minutes         |
| gpt2-small (124M)  | CPU (M3 MacBook Air)  | 5.74 minutes         |
| gpt2-small (124M)  | GPU (M3 MacBook Air)  | 3.73 minutes         |
| gpt2-small (124M)  | GPU (L4)              | 0.69 minutes         |
| gpt2-small (124M)  | GPU (A100)            | 0.39 minutes         |

</div>

Com o modelo e os carregadores de dados preparados, podemos prosseguir com o treinamento do modelo. O código a seguir configura o processo de treinamento, incluindo a inicialização do otimizador, definindo o número de épocas e definindo a frequência de avaliação e o contexto inicial para avaliar as respostas LLM geradas durante o treinamento com base na primeira instrução do conjunto de validação (`val_data[0]`) que examinamos anteriormente:

In [ ]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.1)

num_epochs = 2

train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context=format_input(val_data[0]), tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")


A saída exibe o progresso do treinamento ao longo de duas épocas, onde a diminuição constante das perdas indica uma capacidade crescente de seguir instruções e gerar respostas apropriadas; portanto, o modelo treina bem. (Como o modelo demonstrou aprendizado eficaz nessas duas épocas, estender o treinamento para uma terceira época ou mais não é essencial e pode até ser contraproducente aqui, pois poderia levar a um aumento do sobreajuste.)

Além disso, com base no texto da resposta impresso após cada época, podemos ver que o modelo segue corretamente a instrução de converter a frase de entrada `'The chef cooks the meal every day.'` para voz passiva `'The meal is cooked every day by the chef.'` (Formatar e avaliar as respostas adequadamente mais tarde).

Finalmente, vamos dar uma olhada nas curvas de perda de treinamento e validação:

In [ ]:
from LLM_Definitions import plot_losses

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)


A linha sólida representa a perda de treinamento, mostrando uma diminuição acentuada antes de se estabilizar, enquanto a linha tracejada representa a perda de validação, que segue um padrão semelhante.

Como podemos ver, a perda diminui acentuadamente no início da primeira época, o que significa que o modelo começa a aprender rapidamente. Também podemos ver que um leve sobreajuste se instala por volta de 1 época de treinamento.

## 6. Extraindo e Salvando a Respostas
---


Depois de afinar o LLM na parte de treinamento do conjunto de instruções, agora prosseguimos para avaliar seu desempenho no conjunto de teste reservado. Para isso, primeiro extraímos as respostas geradas pelo modelo para cada entrada do conjunto de teste e as coletamos para análise manual, como ilustrado no panorama apresentado no início deste notebook.

Começamos com o passo 7, a etapa de instrução de resposta, utilizando a função `generate`. A função `generate` retorna o texto combinado (entrada + saída), então usamos fatiamento (`slicing`) e o método `.replace()` sobre o conteúdo da variável `generated_text` para extrair apenas a resposta do modelo. Em seguida, imprimimos as respostas do modelo ao lado das respostas esperadas do conjunto de teste para os três primeiros itens, apresentando-as lado a lado para comparação:


In [ ]:
torch.manual_seed(123)

for entry in test_data[:3]:            # Iterate over the first 3 test set samples

    input_text = format_input(entry)

    token_ids = generate(              # Use the generate function imported earlier
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
)

    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nModel response:\n>> {response_text.strip()}")
    print("-------------------------------------")

Como podemos observar com base nas instruções do conjunto de teste, nas respostas fornecidas e nas respostas do modelo, o desempenho do modelo é relativamente bom. As respostas à primeira e última instrução estão claramente corretas. A segunda resposta está próxima; o modelo responde com “cumulus cloud” em vez de “cumulonimbus” (no entanto, observe que nuvens cumulus podem evoluir para nuvens cumulonimbus, que são capazes de produzir tempestades).

O mais importante é perceber que a avaliação do modelo não é tão direta quanto no notebook anterior, onde bastava calcular a porcentagem de rótulos corretos de spam/não‑spam para obter a acurácia de classificação. Na prática, LLMs finetunados com instruções, como chatbots, são avaliados por múltiplas abordagens:

- **Benchmarks de respostas curtas e escolha múltipla** (ex.: MMLU – “Measuring Massive Multitask Language Understanding”, <https://arxiv.org/abs/2009.03300>), que testam o conhecimento do modelo;

- **Comparação de preferência humana** com outros LLMs, como a arena de chatbots LMSYS (<https://arena.lmsys.org>);

- **Benchmarks conversacionais automatizados**, onde outro LLM (por exemplo, GPT‑4) avalia as respostas, como AlpacaEval (<https://tatsu-lab.github.io/alpaca_eval/>).

Na prática, pode ser útil considerar os três tipos de métodos de avaliação: perguntas de múltipla escolha, avaliação humana e métricas automatizadas que medem o desempenho conversacional. Entretanto, visto que nosso foco principal é avaliar o desempenho em diálogos ao invés da mera capacidade de responder a questões de múltipla escolha, os métodos 2 (avaliação humana) e 3 (métricas automatizadas) podem ser mais relevantes. Considerando a escala da tarefa, implementaremos uma abordagem semelhante à do método 3, que envolve avaliar as respostas automaticamente usando outro LLM. Isso nos permitirá medir eficientemente a qualidade das respostas geradas sem necessidade de envolvimento humano extensivo, economizando tempo e recursos enquanto ainda obtemos indicadores significativos de desempenho.

Na próxima seção, usaremos um procedimento parecido com o AlpacaEval e empregaremos outro LLM para avaliar as respostas do nosso modelo; porém, utilizaremos nosso próprio conjunto de teste ao invés de um dataset público. Isso possibilita uma avaliação mais direcionada e relevante do desempenho do modelo dentro do contexto dos casos de uso pretendidos representados em nosso conjunto de instruções. Para isso, anexamos as respostas geradas pelo modelo ao dicionário `test_data` e salvamos como arquivo `"instruction-data-with-response.json"` para fins de registro, permitindo que carreguemos e analisemos em sessões Python distintas, se necessário.

O código a seguir usa o método `generate` da mesma forma que antes; porém, agora iteramos sobre todo o `test_set`. Além disso, ao invés de imprimir as respostas do modelo, adicionamos elas ao dicionário `test_set`, ou seja, salvamos:

In [ ]:
from tqdm import tqdm   # use the progress bar library

for i, entry in tqdm(enumerate(test_data), total=len(test_data)):

    input_text = format_input(entry)

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = generated_text[len(input_text):].replace("### Response:", "").strip()

    test_data[i]["model_response"] = response_text


with open("instruction-data-with-response.json", "w") as file:
    json.dump(test_data, file, indent=4)  # "indent" for pretty-printing

In [ ]:
print(test_data[0])

Finalmente, salvamos o modelo como `gpt2-medium355M-sft.pth`:

In [ ]:
import re

# Remove white spaces and parentheses from file name
file_name = f"{re.sub(r'[ ()]', '', CHOOSE_MODEL) }-sft.pth"  # sft = supervised finetuning
torch.save(model.state_dict(), file_name)
print(f"Model saved as {file_name}")

# Load the model via
# model.load_state_dict(torch.load("gpt2-medium355M-sft.pth"))

## 7. Avaliando um LLM-AFI
---

Anteriormente, avaliamos o desempenho de um modelo finetunado com instruções examinando suas respostas em 3 exemplos do conjunto de teste. Embora isso nos dê uma ideia aproximada de quão bem o modelo funciona, esse método não escala bem para grandes volumes de respostas. Portanto, nesta seção, automatizamos a avaliação das respostas do LLM finetunado usando outro LLM maior. Em particular, utilizamos um modelo Llama 3 de 8 bilhões de parâmetros finetunado com instruções pela Meta AI que pode ser executado localmente via **ollama** (<https://ollama.com>), e implementamos uma técnica para quantificar o desempenho do modelo finetunado pontuando as respostas geradas pelo teste.

### Ollama

O **ollama** é um aplicativo eficiente para gerenciar e interagir com grandes modelos de linguagem (LLMs) de forma prática, permitindo a execução desses modelos em laptops. Ele funciona como uma camada de abstração sobre a biblioteca open‑source *llama.cpp* (<https://github.com/ggerganov/llama.cpp>), que implementa LLMs em puro C/C++ para maximizar eficiência. Note que o Ollama é apenas uma ferramenta de inferência (geração de texto) e **não** suporta treinamento ou finetuning de LLMs.

Antes de rodar o código abaixo, instale o Ollama visitando <https://ollama.com> e seguindo as instruções (por exemplo, clique em “Download” e baixe a aplicação do Ollama para seu sistema operacional).

Em geral, antes de usar o Ollama via linha de comando, precisamos iniciar a aplicação ou executar `ollama serve` em um terminal separado.

```html
<br/>
<div style='align: left; text-align:center;'>
    <img src='https://camo.githubusercontent.com/01b2c5c8946336c8f500adbdf9eae0b942a5603a0762cbd7f6d65b7f38eba38d/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f32302e776562703f31' alt= '' style="height:450px;"/>
    <br/>
</div>
```

A figura mostra duas opções para executar o Ollama. O painel esquerdo ilustra a inicialização usando `ollama serve`. O painel direito mostra uma segunda opção no macOS, executando a aplicação Ollama em segundo plano ao invés de usar o comando `ollama serve`.

Com a aplicação Ollama ou `ollama serve` rodando em outro terminal, na linha de comando execute:

```bash
# 8B model
ollama run llama3
```

A saída fica assim:

```
$ ollama run llama3
pulling manifest
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B
verifying sha256 digest
writing manifest
removing any unused layers
success
```

> **Obs.:** `llama3` refere-se ao modelo Llama 3 de 8 bilhões de parâmetros finetunado com instruções. Usar o Ollama com o modelo `"llama3"` requer 16 GB de RAM; se seu computador não suportar, você pode tentar um modelo menor, como o phi‑3 de 3.8B, definindo `model = "phi-3"`, que exige apenas 8 GB de RAM. Alternativamente, pode usar o modelo maior Llama 3 de 70 bilhões de parâmetros (se sua máquina suportar), substituindo `llama3` por `llama3:70b`.

Depois da descarga, aparecerá um prompt na linha de comando permitindo conversar com o modelo. Por exemplo:

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered
stomach and eat plants that are high in fiber. In the wild, llamas
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall
grasses, wheat, oats, and barley.
```

> **Obs.:** A resposta pode variar pois o Ollama não é determinístico no momento da escrita.

Você pode encerrar a sessão digitando `/bye`. Contudo, mantenha o comando `ollama serve` ou a aplicação Ollama em execução pelo restante desta etapa.



O código a seguir verifica se a sessão do Ollama está funcionando corretamente antes de prosseguir com a avaliação das respostas geradas no conjunto de teste da seção anterior:


In [ ]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Certifique‑se de que a saída ao executar o código anterior exiba `Ollama running: True`.  
Se aparecer `False`, verifique se o comando `ollama serve` ou o aplicativo Ollama está em execução ativa.

In [ ]:
# This cell is optional; it allows you to restart the notebook
# and only run the previous section without rerunning any of the previous code

import json
from tqdm import tqdm

file_path = "instruction-data-with-response.json"

with open(file_path, "r") as file:
    test_data = json.load(file)


def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

Uma forma alternativa de usar o comando `ollama run` para interagir com o modelo é por meio da sua API REST em Python, usando a função abaixo. Antes de executar as próximas células deste notebook, garanta que o Ollama ainda esteja rodando. A função `query_model` demonstra como utilizar essa API:

In [ ]:
import urllib.request

def query_model(
    prompt,
    model="llama3",
    url="http://localhost:11434/api/chat"
):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048  # context size
        }
    }


    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

# now an example of how to use the query_llama function just implemented
model = "llama3"
result = query_model("What do Llamas eat?", model)
print(result)


Com a função `query_model` definida acima, podemos avaliar as respostas geradas pelo nosso modelo finetunado com um prompt que instrui o Llama 3 a pontuar as respostas do nosso modelo em uma escala de 0 a 100, usando a resposta do conjunto de teste como referência.

Primeiro, aplicamos essa abordagem nos três primeiros exemplos do conjunto de teste que examinamos em uma seção anterior:

In [ ]:
for entry in test_data[:3]:
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"score the model response `{entry['model_response']}`"
        f" on a scale from 0 to 100, where 100 is the best score. "
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model_response"])
    print("\nScore:")
    print(">>", query_model(prompt))
    print("\n-------------------------")


Como podemos observar, o Llama 3 fornece uma avaliação razoável e também atribui pontos parciais quando o modelo não está totalmente correto, como no caso da resposta “cumulus cloud”. Note que o prompt original retorna avaliações muito detalhadas; podemos modificá‑lo para gerar apenas respostas inteiras entre 0 e 100 (onde 100 é a melhor pontuação). Essa modificação permite calcular uma média de pontuação do nosso modelo, oferecendo uma avaliação mais concisa e quantitativa.

A função `generate_model_scores` abaixo usa um prompt alterado que instrui o modelo a “Responder apenas com o número inteiro”. A avaliação dos 110 itens do conjunto de teste leva cerca de 1 minuto em um laptop MacBook Air M3.


In [ ]:
def generate_model_scores(json_data, json_key, model="llama3"):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt, model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Could not convert score: {score}")
            continue

    return scores


scores = generate_model_scores(test_data, "model_response")
print(f"Number of scores: {len(scores)} of {len(test_data)}")
print(f"Average score: {sum(scores)/len(scores):.2f}\n")

Nosso modelo alcança uma pontuação média de aproximadamente **50**, que podemos usar como ponto de referência para comparar com outros modelos ou testar outras configurações de treinamento que possam melhorar o desempenho. Observe que o Ollama não é totalmente determinístico entre sistemas operacionais (até a data desta redação), então os números obtidos podem diferir ligeiramente dos mostrados acima.

Para referência, as pontuações originais são:
- Modelo base Llama 3 8B: **58,51**
- Modelo Instruct Llama 3 8B: **82,65**

Para melhorar ainda mais o desempenho do nosso modelo, podemos explorar várias estratégias, como:

- Ajustar os hiperparâmetros durante o finetuning (taxa de aprendizado, tamanho do lote, número de épocas).
- Aumentar o tamanho ou diversificar o conjunto de treinamento para cobrir uma gama maior de tópicos e estilos.
- Experimentar diferentes prompts ou formatos de instrução que guiem as respostas do modelo de forma mais eficaz.
- Considerar o uso de um modelo pré‑treinado maior, que pode ter maior capacidade de capturar padrões complexos e gerar respostas mais precisas.

## 8. Conclusão
---

Cobrimos as etapas principais do ciclo de desenvolvimento de LLMs: implementar uma arquitetura de LLM, pré‑treinar um LLM e fine‑tune‑á-lo.

Uma etapa opcional que às vezes é seguida após o fine‑tuning por instruções é o *preference finetuning* (fine‑tuning por preferência), que pode ser particularmente útil para personalizar um modelo de modo a se alinhar melhor com preferências específicas do usuário.

Você pode estar interessado em usar LLMs diferentes e mais poderosos para aplicações do mundo real. Para isso, pode considerar ferramentas populares como **axolotl** ([https://github.com/OpenAccess-AI-Collective/axolotl](https://github.com/OpenAccess-AI-Collective/axolotl)) ou **LitGPT** ([https://github.com/Lightning-AI/litgpt](https://github.com/Lightning-AI/litgpt)).

Os campos de IA e pesquisa em LLMs estão evoluindo num ritmo acelerado. Uma forma de acompanhar as últimas novidades é explorar artigos recentes no arXiv em https://arxiv.org/list/cs.LG/recent. Além disso, muitos pesquisadores e profissionais são muito ativos na divulgação e discussão das últimas inovações nas plataformas sociais como X (anteriormente Twitter) e Reddit. O subreddit **r/LocalLLaMA**, em particular, é um bom recurso para se conectar com a comunidade e ficar informado sobre as ferramentas e tendências mais recentes.